In [1]:
from bs4 import BeautifulSoup
import re
import regex
from tqdm import tqdm
from os import listdir
from os.path import join
import nltk
import ast
import sys
import pandas as pd

In [2]:
intentxl=pd.ExcelFile('Voice Banking Demo Examples.xlsx')
intents=intentxl.sheet_names

In [3]:
# General cleaning script, slightly modified for intents
w2npath='/home/hishab/langresource/data/num_conversion_dicts/bn_common_word_2_num.dict'
with open(w2npath,'r') as f1: w2n=ast.literal_eval(f1.read())
sys.path.insert(1, '/home/hishab/asr/training/preprocessing/')
from num_2_standard_bn_words import num_2_word
replace_chars={'য়':'য়','ড়':'ড়','ো':'ো',"৷":" ","ৱ":"র","ৰ":"র",'ৌ':'ৌ',"৺":"","ঌ":"৯",
               "৻":"","৹":"","৥":"","ঽ":"হ","৸":"","৶":"","৤":"",'ৗ':'',"৵":"",'ৣ':"্র",
              "৳":"ট","৅":"","৤":"","৤":"","৴":"","ঢ়":"ঢ়","spn":" "}
def nummaker(obj):
    numchanged=' '+num_2_word(obj.group(1),w2n)+' '
    return numchanged
def clean_intents(uncleanlines):
    cleanlines=[]
    for cleanstep in tqdm(uncleanlines):
        for k,v in replace_chars.items(): cleanstep=str(cleanstep).replace(k,v)
        cleanstep=cleanstep.replace(u'\xa0', u' ')
        for pattern in re.findall(r'\d+\s+-\s+\d+',cleanstep): cleanstep=cleanstep.replace(pattern,pattern.replace('-',' থেকে ')) 
        cleanstep=re.sub("[\(\[].*?[\)\]]", " ",cleanstep) # Remove anything contained within brackets, including the brackets
                                                          # Must be done before removing non-alphabetic characters
        if re.search(r'[a-zA-Z]',cleanstep): # Drop if cleaned sentence contains English characters
            continue
        try:
            cleanstep=re.sub(r'([0-9০-৯]+)',nummaker,cleanstep) # Replace numbers in the string
        except:
            continue # The number is too long. Likely a phone number. Drop this sample entirely. 
        cleanstep=re.sub("%"," শতাংশ ",cleanstep) # Benefit of doubt - let % within the survived sample be percentage
        cleanstep=regex.sub('[^\p{InBengali}\p{Z}]',' ',cleanstep) # Remove non-letters, non-glyphs, and non-spaces
        cleanstep=re.sub(' +', ' ',cleanstep) # Finally, merge multiple spaces
        cleanstep=cleanstep.strip() # Remove leading and trailing whitespaces
        if len(cleanstep): # Cleaned sentence isn't empty
            cleanlines.append('    - '+cleanstep+'\n')
    print("Input: ",len(uncleanlines)," sentences. Output: ",len(cleanlines)," sentences.")
    return cleanlines

In [4]:
with open('CleanIntents.nlu','w') as f:
    for intent in intents:
        intentdf=pd.read_excel('Voice Banking Demo Examples.xlsx',sheet_name=intent)
        intentexamples=intentdf.values[:,0]
        f.write('- intent: '+intent+'\n  examples: |\n')
        cleanexamples=clean_intents(intentexamples)
        f.writelines(cleanexamples)
        f.write('\n')

100%|██████████| 21/21 [00:00<00:00, 28588.25it/s]


Input:  23  sentences. Output:  23  sentences.
Input:  24  sentences. Output:  24  sentences.
Input:  21  sentences. Output:  21  sentences.
Input:  21  sentences. Output:  21  sentences.
Input:  27  sentences. Output:  23  sentences.
Input:  27  sentences. Output:  23  sentences.
Input:  25  sentences. Output:  21  sentences.
Input:  25  sentences. Output:  20  sentences.
Input:  21  sentences. Output:  21  sentences.


100%|██████████| 12/12 [00:00<00:00, 28711.72it/s]

Input:  21  sentences. Output:  21  sentences.
Input:  25  sentences. Output:  22  sentences.
Input:  25  sentences. Output:  22  sentences.
Input:  9  sentences. Output:  9  sentences.
Input:  9  sentences. Output:  9  sentences.
Input:  9  sentences. Output:  9  sentences.
Input:  12  sentences. Output:  12  sentences.
